In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder.appName("app").getOrCreate()

In [4]:
data = [
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
]

schema = ["id","text"]

In [6]:
data = spark.createDataFrame(data, schema)
data.show()

+---+--------------------+
| id|                text|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [ ]:
# data = spark.read.csv("data.csv",inferSchema=True, header=True)
# data.show()

+---+--------------------+
| id|                text|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')

In [8]:
regex_tokenizer = RegexTokenizer(inputCol='text', outputCol='words', pattern='\\W')

In [9]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [10]:
tokenized = tokenizer.transform(data)

In [11]:
tokenized.show()

+---+--------------------+--------------------+
| id|                text|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [12]:
tokenized.withColumn("tokens",count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|                text|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [13]:
rg_tokenized = regex_tokenizer.transform(data)


In [14]:
rg_tokenized.withColumn("tokens",count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|                text|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



### StopWords

In [15]:
from pyspark.ml.feature import StopWordsRemover

In [16]:
data = [(0,["I", "Saw", "the", "green" , "horse"]),
        (1,["Mary", "had", "a", "little", "lamb"])]

schema = ["id", "tokens"]

In [17]:
sentenceDataFrame = spark.createDataFrame(data, schema)

In [18]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, Saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [20]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [21]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, Saw, the, gre...| [Saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [22]:
from pyspark.ml.feature import NGram

In [23]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [25]:
ngrams = NGram(n= 2 , inputCol='words', outputCol='grams')

In [28]:
ngrams.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

